In [1]:
from typing import List, Tuple, Dict, Any, Optional
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))
os.environ["GROQ_API_KEY"] = Path("../groq_token.txt").read_text().strip()

In [60]:
PATH_DOC_STD = Path("../generators/strategies/docs_artifacts/qiskit_std.md")
PATH_DOC_GATES = Path(
    "../generators/strategies/docs_artifacts/qiskit_gates.md")

MAP_REGEX_TO_CHANGELOG = {
    "from qiskit import .*Aer": "Replace from qiskit import Aer with from qiskit_aer import Aer",
    "execute": """
# Legacy path
from qiskit import execute

job = execute(circuit, backend)

# New path
from qiskit import transpile

new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit)
        """,
    "\.qasm\(\)": """
from qiskit import QuantumCircuit
qc = QuantumCircuit(1)
# Old
qasm_str = qc.qasm()
# Alternative
from qiskit.qasm2 import dumps
qasm_str = dumps(qc)
# Alternative: Write to file
from qiskit.qasm2 import dump
with open("my_file.qasm", "w") as f:
    dump(qc, f)
""",
    "from.*import.*CNOT.*": "Replace CNOT with CXGate, both import and use",

}

## Generate and Fix

In [56]:
import re
import dspy
from groq import GroqError
from pathlib import Path
from generators.strategies.llm_generator import LLMGenerationStrategy
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from functools import partial
one_retry = partial(backtrack_handler, max_backtracks=1)


class GenerateNovelQuantumCircuitFromDoc(dspy.Signature):
    """Generate a quantum circuit in Qiskit to generate a novel quantum circuit."""
    documentation: str = dspy.InputField(
        desc="An extract of the documentation of Qiskit")
    inspired_by: str = dspy.InputField(
        desc="The title of the research paper in quantum computing that inspired the circuit")
    python_code: str = dspy.OutputField()


class GenerateLatestQiskitCircuit(dspy.Module):

    def __init__(self, map_regex_to_changelog: dict, *args, **kwargs):
        self.map_regex_to_changelog = map_regex_to_changelog
        self.inspired_by = kwargs.pop("inspired_by", None)
        self.generate_new = dspy.ChainOfThought(
            GenerateNovelQuantumCircuitFromDoc)

    def forward(self, documentation: str) -> str:
        result = self.generate_new(
            documentation=documentation,
            inspired_by=self.inspired_by)
        _python_code = result.python_code
        print(_python_code)
        _code_lines = _python_code.split("\n")
        # check if the old imports are present in the python code
        violations = {}
        for regex_str, changelog in self.map_regex_to_changelog.items():
            regex = re.compile(regex_str)
            # if any(regex.search(line) for line in _code_lines):
            # violations[regex] = changelog
            for line in _code_lines:
                if regex.search(line):
                    print("matching line:", line)
                    violations[regex] = changelog
                    break
        formatted_all_violations = "\n".join(
            f"{regex.pattern}: {changelog}" for regex,
            changelog in violations.items())
        print("Violations")
        print(formatted_all_violations)
        dspy.Suggest(
            not bool(violations),
            f"Please update the imports in the code. The following violations were found:\n{formatted_all_violations}",
            target_module=self.generate_new)
        return result


class LLMGenerationStrategy001(LLMGenerationStrategy):
    """It generates Qiskit circuits using the LLM model."""

    def __init__(
            self,
            path_to_documentation: Path,
            map_regex_to_changelog: dict,
            inspiration_paper_name: str,
            *args, **kwargs):
        super().__init__(path_to_documentation, *args, **kwargs)
        # self.generator = GenerateLatestQiskitCircuit(
        #     map_regex_to_changelog=map_regex_to_changelog,
        #     inspired_by=inspiration_paper_name,
        #     temperature=1.5).activate_assertions()
        self.generator = assert_transform_module(
            GenerateLatestQiskitCircuit(
                map_regex_to_changelog=map_regex_to_changelog,
                inspired_by=inspiration_paper_name,
                temperature=1.5
            ), one_retry)
        self.retries = 0

    def generate(self) -> str:
        """Generates a quantum circuit."""
        self.retries += 1
        try:
            result = self.generator(self.documentation_content)
            python_code = self._clean_tags(result.python_code)
            return python_code
        except GroqError as e:
            if self.retries < 3:
                print(f"Retrying... {self.retries}")
                return self.generate()
            else:
                raise e
        except Exception as e:
            raise e
        return ""

In [52]:
gen001 = LLMGenerationStrategy001(
    path_to_documentation=PATH_DOC_STD,
    inspiration_paper_name="Emerging quantum computing algorithms for quantum chemistry",
    map_regex_to_changelog=MAP_REGEX_TO_CHANGELOG)
new_snippet = gen001.generate()
print(new_snippet)
# dspy.inspect_history(n=4)

2024/12/26 22:56:42 INFO dspy.primitives.assertions: SuggestionFailed: Please update the imports in the code. The following violations were found:
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import execute

job = execute(circuit, backend)

# New path
from qiskit import transpile

new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit)
        
.qasm(): 
from qiskit import QuantumCircuit
qc = QuantumCircuit(1)
# Old
qasm_str = qc.qasm()
# Alternative
from qiskit.qasm2 import dumps
qasm_str = dumps(qc)
# Alternative: Write to file
from qiskit.qasm2 import dump
with open("my_file.qasm", "w") as f:
    dump(qc, f)

2024/12/26 22:56:42 INFO dspy.primitives.assertions: SuggestionFailed: Please update the imports in the code. The following violations were found:
.qasm(): 
from qiskit import QuantumCircuit
qc = QuantumCircuit(1)
# Old
qasm_str = qc.qasm()
# Alternative
from qiskit.qasm2 import dum

```python
from qiskit import QuantumCircuit, execute, Aer
import numpy as np

# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Apply a series of rotations to the qubits
qc.rx(np.pi/2, 0)  # Rotate qubit 0 by pi/2 around the x-axis
qc.ry(np.pi/4, 1)  # Rotate qubit 1 by pi/4 around the y-axis
qc.rz(np.pi/6, 0)  # Rotate qubit 0 by pi/6 around the z-axis

# Apply a controlled-NOT gate to entangle the qubits
qc.cx(0, 1)

# Measure the qubits
qc.measure_all()

# Use the Aer simulator to run the circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator)
result = job.result()

# Print the results
print(result.get_counts())
```
matching line: from qiskit import QuantumCircuit, execute, Aer
matching line: from qiskit import QuantumCircuit, execute, Aer
matching line: simulator = Aer.get_backend('qasm_simulator')
Violations
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import

In [53]:
print(new_snippet)
exec(new_snippet)

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit import transpile
import numpy as np

# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Apply a series of rotations to the qubits
qc.rx(np.pi/2, 0)  # Rotate qubit 0 by pi/2 around the x-axis
qc.ry(np.pi/4, 1)  # Rotate qubit 1 by pi/4 around the y-axis
qc.rz(np.pi/6, 0)  # Rotate qubit 0 by pi/6 around the z-axis

# Apply a controlled-NOT gate to entangle the qubits
qc.cx(0, 1)

# Measure the qubits
qc.measure_all()

# Use the Aer simulator to run the circuit
simulator = Aer.get_backend('qasm_simulator')
new_circuit = transpile(qc, simulator)
job = simulator.run(new_circuit)
result = job.result()

# Print the results
print(result.get_counts())

# Get the QASM string of the circuit
from qiskit.qasm2 import dumps
qasm_str = dumps(qc)
print(qasm_str)
{'11': 437, '01': 87, '10': 80, '00': 420}
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg meas[2];
rx(pi/2) q[0];
ry(pi/4) q[1];
rz(pi/6) q[0];
cx 

In [28]:
import dspy


class MakeGreeting2(dspy.Module):
    def __init__(self, invalid_greetings=[]):
        self.invalid_greetings = invalid_greetings
        self.prog = dspy.ChainOfThought("context -> greeting")

    def forward(self, context):
        result = self.prog(context=context)
        _greeting = result.greeting
        print(_greeting)
        greeting_violations = list(
            filter(
                lambda x: x.lower() in
                _greeting.lower(),
                self.invalid_greetings))
        print(greeting_violations)
        formatted = ", ".join(greeting_violations)
        dspy.Suggest(
            not bool(greeting_violations),
            f"Greetings like {formatted} are so bad, provide a different greeting.",
            target_module=self.prog)
        return result

## Simple test

In [33]:
g2 = MakeGreeting2(invalid_greetings=['hello']).activate_assertions()
context = "Provide a greeting!"
g2.forward(context)
dspy.inspect_history(n=2)

2024/12/26 14:49:55 INFO dspy.primitives.assertions: SuggestionFailed: Greetings like hello are so bad, provide a different greeting.


Hello! It's nice to meet you.
['hello']
Greetings! It's a pleasure to interact with you.
[]




[2024-12-26T14:49:55.443216]

System message:

Your input fields are:
1. `context` (str)

Your output fields are:
1. `reasoning` (str)
2. `greeting` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## greeting ## ]]
{greeting}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, produce the fields `greeting`.


User message:

[[ ## context ## ]]
Provide a greeting!

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## greeting ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The task requires generating a greeting based on the provided context, which is to provide a greeting. Since there's no specific inf

# Generate Diversity

In [58]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


class Generate20QuantumPaperTitles(dspy.Signature):
    """Generate 20 paper titles for potential research papers in quantum computing."""
    inspiration_word: str = dspy.InputField(
        desc="A word that inspires the paper titles")
    paper_title: List[str] = dspy.OutputField()


class GeneratePaperTitles(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought(Generate20QuantumPaperTitles)

    def forward(self, inspiration_word) -> Dict[str, Any]:
        results = self.prog(inspiration_word=inspiration_word)
        return results


def get_unique_words_from_titles(titles: List[str]) -> List[str]:
    # Tokenize the titles and remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    words = [
        word.lower()
        for title in titles
        for word in word_tokenize(title)
        if word.lower() not in stop_words and word not in punctuation
    ]

    # Get unique words
    return list(set(words))


inspiration_words = ["quantum"]
paper_titles = []

max_rounds = 3
current_round = 0

while len(inspiration_words) > 0 and current_round < max_rounds:

    inspiration_word = inspiration_words.pop(0)

    # generate 20 titles
    gen_paper_titles = GeneratePaperTitles()
    res = gen_paper_titles.forward(inspiration_word=inspiration_word)
    new_paper_titles = res.paper_title
    paper_titles.extend(new_paper_titles)
    # new words to inspire the next round, split all titles in words
    new_words = get_unique_words_from_titles(new_paper_titles)
    inspiration_words.extend(new_words)
    print(f"Round {current_round} - Inspiration word: {inspiration_word}")
    print(new_paper_titles)
    current_round += 1

Round 0 - Inspiration word: quantum
['Quantum Computing for Solving Complex Optimization Problems', 'A Quantum Approach to Machine Learning', 'Quantum Error Correction Codes for Reliable Quantum Computing', 'Quantum Simulation of Chemical Reactions', 'Quantum Algorithms for Factoring Large Numbers', 'Quantum Computing for Materials Science Applications', 'A Survey of Quantum Machine Learning Algorithms', 'Quantum Computing for Cybersecurity', 'Quantum Information Processing with Superconducting Qubits', 'Quantum Computing for Solving Linear Algebra Problems', 'Quantum Computing and Its Applications in Chemistry', 'Quantum Error Correction with Topological Codes', 'A Quantum Approach to Clustering Algorithms', 'Quantum Computing for Image Recognition', 'Quantum Machine Learning for Natural Language Processing', 'Quantum Computing for Solving Differential Equations', 'Quantum Algorithms for Computational Biology', 'Quantum Computing for Financial Modeling', 'Quantum Computing and Its App

In [66]:
new_snippets = []

n_snippets = 3
current_round = 0

while len(new_snippets) < n_snippets:

    # pick inspiring title
    inspiration_title = paper_titles.pop(0)
    print(f"Round {current_round} - Inspiration title: {inspiration_title}")

    gen001 = LLMGenerationStrategy001(
        path_to_documentation=PATH_DOC_GATES,
        inspiration_paper_name=inspiration_title,
        map_regex_to_changelog=MAP_REGEX_TO_CHANGELOG)
    new_snippet = gen001.generate()
    new_snippets.append(new_snippet)
    import time
    time.sleep(60)

    current_round += 1

Round 0 - Inspiration title: Quantum Error Correction Codes for Reliable Quantum Computing


2024/12/26 23:04:43 INFO dspy.primitives.assertions: SuggestionFailed: Please update the imports in the code. The following violations were found:
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import execute

job = execute(circuit, backend)

# New path
from qiskit import transpile

new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit)
        


```python
from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit.library import XGate, CXGate

# Create a quantum circuit with 3 qubits and 3 classical bits
qc = QuantumCircuit(3, 3)

# Encode the qubit (assuming the first qubit is the data qubit)
qc.barrier()
qc.cx(0, 1)
qc.cx(0, 2)
qc.barrier()

# Apply a CNOT gate to simulate an error on the second qubit
qc.cx(0, 1)

# Apply the correction circuit
qc.barrier()
qc.ccx(1, 2, 0)
qc.barrier()

# Measure the qubits
qc.measure([0, 1, 2], [0, 1, 2])

# Print the circuit
print(qc.draw())

# Run the circuit on a simulator
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)
print(counts)
```
matching line: from qiskit import QuantumCircuit, execute, Aer
matching line: from qiskit import QuantumCircuit, execute, Aer
Violations
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import

2024/12/26 23:05:50 INFO dspy.primitives.assertions: SuggestionFailed: Please update the imports in the code. The following violations were found:
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import execute

job = execute(circuit, backend)

# New path
from qiskit import transpile

new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit)
        


```python
from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit.library import XGate, CNOT, RYGate

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Apply a rotation gate to the first qubit to simulate the initial state of a particle
qc.append(RYGate(3.14/2), [0])

# Apply a controlled-X gate to simulate the interaction between two particles
qc.append(CNOT, [0, 1])

# Apply another rotation gate to the second qubit to simulate the evolution of the system
qc.append(RYGate(3.14/4), [1])

# Apply a controlled-X gate to simulate the interaction between the second and third particles
qc.append(CNOT, [1, 2])

# Apply a final rotation gate to the third qubit to simulate the final state of the system
qc.append(RYGate(3.14/2), [2])

# Print the circuit
print(qc.draw())

# Simulate the circuit using the Aer simulator
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)
print(counts)
```
mat

2024/12/26 23:06:56 INFO dspy.primitives.assertions: SuggestionFailed: Please update the imports in the code. The following violations were found:
from qiskit import .*Aer: Replace from qiskit import Aer with from qiskit_aer import Aer
execute: 
# Legacy path
from qiskit import execute

job = execute(circuit, backend)

# New path
from qiskit import transpile

new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit)
        


```python
from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit.library import XGate, HGate, CXGate

# Create a quantum circuit with 4 qubits and 4 classical bits
qc = QuantumCircuit(4, 4)

# Apply Hadamard gates to the first two qubits to create a superposition
qc.h(0)
qc.h(1)

# Apply a controlled-NOT gate between the first and second qubits for entanglement
qc.cx(0, 2)

# Apply another controlled-NOT gate between the second and third qubits
qc.cx(1, 3)

# Apply a rotation gate to the first qubit
qc.rz(3.14/2, 0)  # Example rotation, actual value depends on the specific algorithm

# Measure the qubits
qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

# Use the Aer simulator to run the circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)

# Get the results
result = job.result()
counts = result.get_counts(qc)

print(counts)
```
matching line: from qiskit import QuantumCircuit, execute, Aer
matching line: from qiskit import QuantumCircuit, e

In [ ]:
for i in range(n_snippets):
    print("=" * 80)
    print(f"Snippet {i+1}")
    print(new_snippets[i])
    print("-" * 80)
    try:
        exec(new_snippets[i])
    except Exception as e:
        print(e)
        pass
    print("=" * 80)

Snippet 1
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, CXGate
from qiskit_aer import Aer

# Create a quantum circuit with 3 qubits and 3 classical bits
qc = QuantumCircuit(3, 3)

# Encode the qubit (assuming the first qubit is the data qubit)
qc.barrier()
qc.cx(0, 1)
qc.cx(0, 2)
qc.barrier()

# Apply a CNOT gate to simulate an error on the second qubit
qc.cx(0, 1)

# Apply the correction circuit
qc.barrier()
qc.ccx(1, 2, 0)
qc.barrier()

# Measure the qubits
qc.measure([0, 1, 2], [0, 1, 2])

# Print the circuit
print(qc.draw())

# Run the circuit on a simulator
simulator = Aer.get_backend('qasm_simulator')
new_circuit = QuantumCircuit.transpile(qc, simulator)
job = simulator.run(new_circuit)
result = job.result()
counts = result.get_counts(qc)
print(counts)
--------------------------------------------------------------------------------
      ░            ░       ░ ┌───┐ ░ ┌─┐      
q_0: ─░───■────■───░───■───░─┤ X ├─░─┤M├──────
      ░ ┌─┴─┐  │   ░ ┌─┴─┐

## Generate 5 Circuits

In [ ]:
class GenerateMultipleCircuit(dspy.Signature):
    """Generate multiple quantum circuits in Qiskit.

    Each circuit:
    - has enough classical bits to measure all qubits
    - has unusual gates
    - registers are referenced with the number only
    - includes intermediate measurements
    - is valid Qiskit python code (statements end with a newline)

    The circuits must have only circuit objects and no execution or transpilation."""
    n_circuits: int = dspy.InputField(
        desc="The number of circuits to generate")
    max_qubits: int = dspy.InputField(
        desc="The maximum number of qubits to use in each circuit")
    n_ops_per_circuit: int = dspy.InputField(
        desc="The number of operations to include in each circuit")
    circuits: List[str] = dspy.OutputField()


class GenerateMultipleCircuits(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought(
            GenerateMultipleCircuit, temperature=1)

    def forward(self, n_circuits: int, max_qubits: int, n_ops_per_circuit: int) -> Dict[str, Any]:
        results = self.prog(n_circuits=n_circuits,
                            max_qubits=max_qubits,
                            n_ops_per_circuit=n_ops_per_circuit)
        return results


res = GenerateMultipleCircuits().forward(
    n_circuits=3,
    n_ops_per_circuit=12,
    max_qubits=12)

print(res)
for i, circuit in enumerate(res['circuits']):
    print(f"Circuit {i+1}")
    print(circuit)
    exec(circuit)
    print(qc.draw())
    print("-" * 80)

Prediction(
    reasoning='To solve this problem, we will generate 3 quantum circuits with a maximum of 12 qubits each. Each circuit will include 12 operations, consisting of a mix of standard gates and measurements. The qubits and classical bits in each circuit will be referenced by their respective numbers. Intermediate measurements will be included to comply with the requirements.',
    circuits=['from qiskit import QuantumCircuit\nqc = QuantumCircuit(6, 6)\nqc.h(0)\nqc.cx(0, 1)\nqc.measure(1, 1)\nqc.x(2)\nqc.barrier()\nqc.measure(2, 2)\nqc.y(3)\nqc.measure(3, 3)\nqc.z(4)\nqc.measure(4, 4)\nqc.s(5)\nqc.measure(5, 5)\n', 'from qiskit import QuantumCircuit\nqc = QuantumCircuit(8, 8)\nqc.x(0)\nqc.barrier()\nqc.measure(0, 0)\nqc.y(1)\nqc.cx(1, 2)\nqc.measure(2, 2)\nqc.h(3)\nqc.measure(3, 3)\nqc.s(4)\nqc.barrier()\nqc.measure(4, 4)\nqc.t(5)\nqc.measure(5, 5)\nqc.measure(6, 6)\nqc.measure(7, 7)\n', 'from qiskit import QuantumCircuit\nqc = QuantumCircuit(10, 10)\nqc.h(0)\nqc.barrier()\nqc.